# Sydney Livability Group Assignment

## Marking Rubric
- Data Import (0.5) : all 5 provided and linked files loaded
- DB Schema (1.5): good normalised schema with al PKs and suitable FKs, consistent naming and suitable domain types (data is interpreted correctly, dates aren't read as strings etc.)
- Neighbourhood Join (2): correct precise matching use PostGis and exact ABS boundaries
- Own Dataset (2): Two or more extra datasets imported into DBMS with good schema of a type not used so far (e.g. JSON) and at least one spatial. 
- Livability Score (3): Livability score well implemented for both Greater Sydney adn City of Sydney, including extension of own datasets were integrated. 
- Indexing (1): more than one useful index created; at least one spatial index.
- Correlation Tests (1): score correlated to BOTH median income and median rent prices
- Dataset Description (1): all data sources briefly described
- Database Description (1) : db schema diagram given
- Liveability Analysis (1) : scoring function well described; good discusion of results ideally with graphical visualisation.
- Correlation analysis (1.5) : correlation analysis described with statistical interpretation
- Data Visualisation (2): good figures/plots plus some useful map-overlay visualisation (optional: even interactive)
- Report Quality (1.5): great report which is in length limit, well structured, well written, no spelling mistakes and grammatically correct, good use of figures to visualise points, good font size and sapcing.
- City of Syney Stakeholder Analysis (1 > 0): Proposed a stakeholder, a brief introduction and showed how you tailored your score for their needs. Results suitable visualised on a map.

## To Do:
- Clean tables. Make sure data is valid for when we upload it to the Sydney_Livability schema. 
- Figure out how to upload the zipped folders of data (non-csv data) into the notebook. 
- Identify the Primary and Foreign Keys for each table.
- Upload the tables into the schema.
- Figure out how to share the schema. Something to do with adding other peoples credentials into the credentials.json folder.
- Identify other databases we want to use from the website provided.
- Identify other stakeholders
- Canvas "Quiz" on stakeholders and additional data: due Week 11 Friday. 

## Notes
Accessing a PostgreSQL database within Python requires psycopg2 and sqlalchemy modules. 
Also require pandas. 
You need a crednetials.json file in the same folder to store database credentials. This will also allow us to share notebooks between users without security concerns and allow multiple credentials to be stored without greatly modifying the notebook. 
We have to use the public schema as it is the only schema with PostGis installed. We do not create our own schema. Will try to figure out how to share tables on a public schema.
This code is from the Week 4 Tutorial

## Connect to the database:

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData


import psycopg2
import psycopg2.extras
import json
import os

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

In [50]:
db, conn = pgconnect(credentials)

Connected successfully.


In [51]:
conn.close()

## Exploring the Schema on PGAdmin:

In [36]:
from sqlalchemy import inspect
inspect(db).get_schema_names()

OperationalError: (psycopg2.OperationalError) connection to server at "soitpw11d59.shared.sydney.edu.au" (10.87.14.236), port 5432 failed: FATAL:  too many connections for database "y22s1d2x01_igro0534"

(Background on this error at: https://sqlalche.me/e/14/e3q8)

Inspect a specific schema:

In [11]:
inspect(db).get_table_names(schema='nswfuel')
inspect(db).get_columns('observations', schema='nswfuel')

[{'name': 'servicestation',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'observationno',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'pricedate',
  'type': DATE(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'pricetime',
  'type': TIME(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None}]

## Create a new schema!

This will set all of the queries you execute to look at the sydney_livability schema. Probably won't return anything useful until you have a sydney_livability schema in your PGAdmin Server. First need to figure out how to share a server with others. 

In [12]:
conn.execute("set search_path to public")

Defining helper function: Query

In [13]:
def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [ ]:
# To switch off converting everything into a pandas dataframe use df = False like so:
# query(conn, "select count(*) from Fuel", df=False)

## Exploring tables provided

In [54]:
nhdata = pd.read_csv('Neighbourhoods.csv')
nhdata.head(50)

,Unnamed: 0,area_id,area_name,land_area,population,number_of_dwellings,number_of_businesses,median_annual_household_income,avg_monthly_rent,0-4,5-9,10-14,15-19
0,0,102011028,Avoca Beach - Copacabana,643.8,7590,2325,738.0,46996.0,1906.0,467,583,604,560
1,1,102011029,Box Head - MacMasters Beach,3208.6,10986,3847,907.0,42621.0,1682.0,586,696,661,692
2,2,102011030,Calga - Kulnura,76795.1,4841,1575,1102.0,42105.0,1182.0,220,254,304,320
3,3,102011031,Erina - Green Point,3379.3,14237,4450,1666.0,43481.0,1595.0,695,778,916,838
4,4,102011032,Gosford - Springfield,1691.2,19385,6373,2126.0,45972.0,1382.0,1200,1079,963,977
5,5,102011033,Kariong,830.6,6540,1824,352.0,47896.0,1649.0,472,502,497,591
6,6,102011034,Kincumber - Picketts Valley,1208.9,7428,1946,479.0,41642.0,1397.0,404,458,448,416
7,7,102011035,Narara,770.2,6853,2006,292.0,46660.0,1512.0,463,470,448,459
8,8,102011036,Niagara Park - Lisarow,1673.2,8173,2345,477.0,45762.0,1495.0,525,605,617,606
9,9,102011037,Point Clare - Koolewong,678.7,6281,2220,312.0,45966.0,1572.0,371,412,373,343


In [55]:
business_stats_data = pd.read_csv('BusinessStats.csv')
business_stats_data.head(50)

,area_id,area_name,number_of_businesses,accommodation_and_food_services,retail_trade,agriculture_forestry_and_fishing,health_care_and_social_assistance,public_administration_and_safety,transport_postal_and_warehousing
0,101021007,Braidwood,629,26,27,280,11,0,35
1,101021008,Karabar,326,7,10,8,11,0,43
2,101021009,Queanbeyan,724,52,47,11,56,3,77
3,101021010,Queanbeyan - East,580,16,23,4,12,0,57
4,101021011,Queanbeyan Region,1642,39,63,292,34,7,81
5,101021012,Queanbeyan West - Jerrabomberra,885,14,48,14,29,3,59
6,101031013,Bombala,419,18,14,250,9,0,14
7,101031014,Cooma,714,55,47,110,36,0,49
8,101031015,Cooma Region,478,11,15,258,5,0,26
9,101031016,Jindabyne - Berridale,1032,144,59,161,43,3,27


## Clean data provided
- Check for 0 or negative values
- Check for null values
- Check pandas is interpreting columns as the correct data type e.g. date is not an integer or string, its a date type.
- Type and name conversion
- Filtering missing data
- Unifying semantic representations
- Matching of entries
- Handling of dates and time

## Load data into new database:

In [43]:
# Create the Neighbourhoods table:

# conn.execute("""
# DROP TABLE IF EXISTS Neighbourhoods;
# CREATE TABLE Neighbourhoods(
#     area_id INTEGER NOT NULL PRIMARY KEY,
#     area_name VARCHAR(50),
#     land_area FLOAT8,
#     population INTEGER,
#     number_of_dwellings INTEGER,
#     number_of_businesses FLOAT8,
#     median_annual_household_income FLOAT8,
#     avg_monthly_rent FLOAT8,
#     child0_4 INTEGER,
#     child5_9 INTEGER,
#     child10_14 INTEGER,
#     child15_19 INTEGER
#     )""")

In [41]:
# DO NOT RUN YET. FIRST MUST CLEAN THE DATA SO IT WILL UPLOAD CORRECTLY. 
# nhdata.to_sql("neighbourhoods", con=conn, if_exists='append', index=False)
# query(conn, "select * from neighbourhoods")

In [42]:
# Create the BusinessStats table:

# conn.execute("""
# DROP TABLE IF EXISTS BusinessStats
# CREATE TABLE BusinessStats(
#     area_id INTEGER NOT NULL REFERENCES BusinessStats(area_id),
#     area_name VARCHAR(50) PRIMARY KEY,
#     number_of_businesses INTEGER,
#     accommodation_and_food_services INTEGER,
#     retail_trade INTEGER,
#     agriculture_forestry_and_fishing INTEGER,
#     health_care_and_social_assistance INTEGER,
#     public_administration_and_safety INTEGER,
#     transport_postal_and_warehousing INTEGER
    
#     )""")

In [ ]:
# DO NOT RUN YET. FIRST MUST CLEAN THE DATA SO IT WILL UPLOAD CORRECTLY. 
# business_stats.to_sql("business_stats", con=conn, if_exists='append', index=False)
# query(conn, "select * from business_stats")

In [ ]:
# Create the SA2_2016_AUST table:

# conn.execute("""
# DROP TABLE IF EXISTS SA2_2016_AUST;
# CREATE TABLE SA2_2016_AUST(
#     SA2_MAIN16 INTEGER NOT NULL PRIMARY KEY,
#     SA2_NAME16 VARCHAR(50) NOT NULL REFERENCES BusinessStats(area_name),
#     SA3_CODE16 INTEGER,
#     SA3_NAME16 VARCHAR(50),
#     STE_CODE16 INTEGER,
#     STE_CODE16 VARCHAR(50),
#     AREASQKM16 FLOAT,
#     geometry GEOMETRY(MULTIPOLYGON,4326)
#     
#     )""")

In [ ]:
# DO NOT RUN YET. FIRST MUST CLEAN THE DATA SO IT WILL UPLOAD CORRECTLY. 
# SA2_2016_AUST.to_sql("SA2_2016_Aust", con=conn, if_exists='append', index=False)
# query(conn, "select * from SA2_2016_Aust")

In [ ]:
# Repeat for break and enter, school chatchments, and the two extra datasets.

In [27]:
# CODE TO DELETE TABLE:
# conn.execute("""
# DROP TABLE IF EXISTS neighbourhoods;
# """)

AttributeError: 'NoneType' object has no attribute 'close'

## Loading GeoData

In [45]:
# Installing the relevant packages
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

In [46]:
break_enter = gpd.read_file("break_and_enter/BreakEnterDwelling_JanToDec2021.shp")
break_enter

,OBJECTID,Contour,Density,ORIG_FID,Shape_Leng,Shape_Area,geometry
0,1,8.0,Low Density,1,0.012138,0.000006,"POLYGON ((149.91078 -37.06636, 149.91080 -37.0..."
1,2,8.0,Low Density,1,0.019106,0.000015,"POLYGON ((149.90601 -37.05837, 149.90602 -37.0..."
2,3,8.0,Low Density,1,0.006068,0.000002,"POLYGON ((148.94250 -37.04209, 148.94253 -37.0..."
3,4,8.0,Low Density,1,0.011006,0.000006,"POLYGON ((149.19583 -36.91833, 149.19584 -36.9..."
4,5,8.0,Low Density,1,0.008073,0.000003,"POLYGON ((149.84836 -36.68459, 149.84838 -36.6..."
...,...,...,...,...,...,...,...
2589,2590,10.0,High Density,3,0.015358,0.000011,"POLYGON ((153.27961 -28.55388, 153.27964 -28.5..."
2590,2591,10.0,High Density,3,0.028327,0.000029,"POLYGON ((150.35939 -28.60530, 150.35941 -28.6..."
2591,2592,10.0,High Density,3,0.006654,0.000002,"POLYGON ((153.56825 -28.38307, 153.56828 -28.3..."
2592,2593,10.0,High Density,3,0.047098,0.000044,"POLYGON ((153.53917 -28.18858, 153.53920 -28.1..."


In [57]:
SA2 = gpd.read_file("SA2_2016_Aust/SA2_2016_AUST.shp")
SA2.head(50)
SA2.tail(50)

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geometry
2260,801071079,81079,Gowrie (ACT),80107,Tuggeranong,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,1.9317,"POLYGON ((149.10274 -35.40864, 149.10268 -35.4..."
2261,801071080,81080,Greenway,80107,Tuggeranong,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,5.3070,"POLYGON ((149.06490 -35.40274, 149.06495 -35.4..."
2262,801071081,81081,Isabella Plains,80107,Tuggeranong,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,2.4950,"POLYGON ((149.08364 -35.42309, 149.08537 -35.4..."
2263,801071082,81082,Kambah,80107,Tuggeranong,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,11.3361,"POLYGON ((149.08049 -35.38349, 149.08062 -35.3..."
2264,801071083,81083,Macarthur,80107,Tuggeranong,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,1.3014,"POLYGON ((149.12509 -35.40573, 149.12401 -35.4..."
2265,801071084,81084,Monash,80107,Tuggeranong,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,3.4119,"POLYGON ((149.08144 -35.41388, 149.08154 -35.4..."
2266,801071085,81085,Mount Taylor,80107,Tuggeranong,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,10.9052,"POLYGON ((149.05912 -35.36942, 149.05340 -35.3..."
2267,801071086,81086,Oxley (ACT),80107,Tuggeranong,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,1.0865,"POLYGON ((149.07467 -35.40457, 149.07469 -35.4..."
2268,801071087,81087,Richardson,80107,Tuggeranong,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,2.2371,"POLYGON ((149.10333 -35.42080, 149.10411 -35.4..."
2269,801071088,81088,Theodore,80107,Tuggeranong,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,3.1353,"POLYGON ((149.11164 -35.44559, 149.11198 -35.4..."


In [48]:
school_catchments_future = gpd.read_file("school_catchments/catchments_future.shp")
school_catchments_primary = gpd.read_file("school_catchments/catchments_primary.shp")
school_catchments_secondary = gpd.read_file("school_catchments/catchments_secondary.shp")
school_catchments_future
school_catchments_primary
school_catchments_secondary

,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geometry
0,8503,HIGH_COED,Billabong HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((146.67182 -35.31444, 146.68930 -35.3..."
1,8266,HIGH_COED,James Fallon HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((147.08734 -35.86271, 147.10413 -35.8..."
2,8505,HIGH_COED,Murray HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((146.81448 -35.78341, 146.81250 -35.7..."
3,8246,HIGH_COED,Heathcote HS,None,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((151.03281 -34.02521, 151.03267 -34.0..."
4,8458,HIGH_COED,Kingswood HS,20201016,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"MULTIPOLYGON (((150.68600 -33.74031, 150.68631..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,8213,HIGH_BOYS,Birrong BHS,20211221,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((151.05364 -33.85076, 151.06142 -33.8..."
431,8857,HIGH_GIRLS,GRC Penshurst Girls,20220110,N,N,N,N,N,N,N,Y,Y,Y,Y,N,N,None,"POLYGON ((151.07573 -33.95089, 151.07921 -33.9..."
432,3235,CENTRAL_HIGH,Tooleybuc CS,20200512,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((143.37723 -34.80173, 143.39037 -34.8..."
433,1115,CENTRAL_HIGH,Balranald CS,20200512,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((143.65541 -33.55702, 143.65541 -33.5..."


In [ ]:
# To convert longitdue and latitude into points example from tutorial:
# cities['geom'] = gpd.points_from_xy(cities.lng, cities.lat)  # creating the geometry column
# cities = cities.drop(columns=['lat', 'lng'])  # removing the old latitude/longitude fields

## Format of the Schema (Primary & Foreign Keys)

BusinessStats(area_id) <-> Neighbourhoods(area_id)
BusinessStats(area_name) <-> SA2(SA2_NAME16)
SA2(geometry) <-> break_enter(geometry)
school_catchments(geometry) <-> SA2(geometry)

